In [1]:
!pip install requests-oauthlib

In [11]:
from requests_oauthlib import OAuth1Session
import os
import json
import pandas as pd

In [5]:
request_token_url = "https://api.twitter.com/oauth/request_token"
oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

try:
    fetch_response = oauth.fetch_request_token(request_token_url)
except ValueError:
    print("There may have been an issue with the consumer_key or consumer_secret you entered.")

In [6]:
resource_owner_key = fetch_response.get("oauth_token")
resource_owner_secret = fetch_response.get("oauth_token_secret")
print("Got OAuth token: %s" % resource_owner_key)

Got OAuth token: 6tIUDQAAAAABKcqLAAABdkAT63M


In [8]:
base_authorization_url = "https://api.twitter.com/oauth/authorize"
authorization_url = oauth.authorization_url(base_authorization_url)
print("Please go here and authorize: %s" % authorization_url)
verifier = input("Paste the PIN here: ")

Please go here and authorize: https://api.twitter.com/oauth/authorize?oauth_token=6tIUDQAAAAABKcqLAAABdkAT63M


Paste the PIN here:  0297752


In [9]:
access_token_url = "https://api.twitter.com/oauth/access_token"
oauth = OAuth1Session(
    consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=resource_owner_key,
    resource_owner_secret=resource_owner_secret,
    verifier=verifier,
)
oauth_tokens = oauth.fetch_access_token(access_token_url)


access_token = oauth_tokens["oauth_token"]
access_token_secret = oauth_tokens["oauth_token_secret"]

In [32]:
df = pd.read_csv('journalist_list.csv')
df.tail()

,Journalist,Publisher
5549,"Zampa, Peter",Gray Television
5550,"Zang, Guohua",CTIŒTV (Taiwan)
5551,"Zarinko, Joseph",CGTN America
5552,"Zayas, Ramon",WRCŒTV/NBCŒ4
5553,"Zechar, David",ABC News


In [53]:
full_name = df['Journalist'].map(lambda x: x.split(', '))

for i, name in enumerate(full_name):
    if len(name) < 2:
        print(i, name)
        
print(df.shape)        
df = df[(df['Journalist'] != 'Albright Nicole Gaudiano') &
       (df['Journalist'] != 'Fernandez-Pereda,Christina') &
        (df['Journalist'] != 'Roland Neil') &
        (df['Journalist'] != 'Baca. Nathan') &
        (df['Journalist'] != 'Kay Katharine')]
print(df.shape)

(5549, 2)
(5549, 2)


In [58]:
df['first name'] = df['Journalist'].map(lambda x: x.split(', ')[1].lower())
df['last name'] = df['Journalist'].map(lambda x: x.split(', ')[0].lower())
df.head()

,Journalist,Publisher,first name,last name
0,"Abrahams-Gray, Rosalie",Atlantic Monthly,rosalie,abrahams-gray
1,"Adragna, Anthony",Politico,anthony,adragna
2,"Aftab, Mirza",BNA News,mirza,aftab
3,"Akin, Stephanie",Roll Call,stephanie,akin
4,"Albergo, Paul",BNA News,paul,albergo


In [59]:
df['twitter_username_1'] = df['first name'].map(str) + df['last name'].map(str)
df['twitter_username_2'] = df['first name'].map(lambda x: x[0]) + df['last name'].map(str)
df['twitter_username_3'] = df['first name'].map(str) + df['last name'].map(lambda x: x[0])
df.head()

,Journalist,Publisher,first name,last name,twitter_username_1,twitter_username_2,twitter_username_3
0,"Abrahams-Gray, Rosalie",Atlantic Monthly,rosalie,abrahams-gray,rosalieabrahams-gray,rabrahams-gray,rosaliea
1,"Adragna, Anthony",Politico,anthony,adragna,anthonyadragna,aadragna,anthonya
2,"Aftab, Mirza",BNA News,mirza,aftab,mirzaaftab,maftab,mirzaa
3,"Akin, Stephanie",Roll Call,stephanie,akin,stephanieakin,sakin,stephaniea
4,"Albergo, Paul",BNA News,paul,albergo,paulalbergo,palbergo,paula


In [ ]:
#AnthonyAdragna
#stephanieakin
#palbergo
#PeterZampa

In [69]:
fields = "created_at,description"
params = {"usernames": "AnthonyAdragna", "user.fields": fields}

In [76]:
oauth = OAuth1Session(consumer_key,
                    client_secret=consumer_secret,
                    resource_owner_key=access_token,
                    resource_owner_secret=access_token_secret,
                    )

response = oauth.get("https://api.twitter.com/2/users/by", params=params)

if response.status_code != 200:
    raise Exception("Request returned an error: {} {}".format(response.status_code, response.text))

print("Response code: {}".format(response.status_code))
json_response = response.json()
print(json.dumps(json_response, indent=4, sort_keys=True))

response = json_response['data'][0]

bio = response['description']
id_ = response['id']

Response code: 200
{
    "data": [
        {
            "description": "Congressional climate and energy issues for @POLITICO. Former @morning_energy scribe. D.C. native. Standard warnings apply. DMs open. Open to story ideas!",
            "id": "460246990",
            "name": "Anthony Adragna",
            "username": "AnthonyAdragna"
        }
    ]
}


In [85]:
def twitter_username(username):
    params = {"usernames": username, "user.fields": "description"}
    
    oauth = OAuth1Session(consumer_key,
                    client_secret=consumer_secret,
                    resource_owner_key=access_token,
                    resource_owner_secret=access_token_secret,
                    )

    response = oauth.get("https://api.twitter.com/2/users/by", params=params)

    if response.status_code == 200:

        response = json_response['data'][0]

        bio = response['description']
        id_ = response['id']

        return pd.DataFrame({'submitted username': [username],
                            'bio': [bio],
                            'twitter id': [id_]})

In [ ]:
twitter_dfs = []
for i in list(range(1, 4)):
    twitter = []
    for user in list(df['twitter_username_{}'.format(i)]):
        twitter.append(twitter_username(name))

    twitter = pd.concat(twitter)
    df_twitter = pd.merge(df, twitter, how='inner', left_on='twitter_username_{}'.format(i), right_on='submitted username')
    
    twitter_dfs.append(df_twitter)
    
twitter_dfs = pd.concat(twitter_dfs)
print(twitter_dfs.shape)
twitter_dfs.head()